# 1. Введение
✍ Из предыдущих модулей вы узнали, как получать данные, когда они хранятся в файлах разных форматов. В реальных проектах данные, как правило, не содержатся в готовом виде в каком-то файле — для анализа необходимо предварительно их собрать, например скопировав с какого-нибудь стороннего веб-сайта.

В этом модуле мы рассмотрим, как с помощью Python можно упростить сбор информации из интернета и, в некоторых случаях, получить доступ к данным, которые невозможно «собрать руками». В частности, мы:

- рассмотрим, в каком виде информация обычно находится в интернете;
- напишем программы, которые собирают данные с веб-сайтов, используя библиотеки requests и BeautifulSoup;
- разберёмся, что такое программный интерфейс веб-сервисов (API) и как его использовать (для примера мы будем собирать данные в социальной сети ВКонтакте).

К концу изучения модуля вы сможете писать программы, которые избавят вас от многих часов монотонной работы и расширят спектр собираемой информации.

# 2. Веб-запросы
Процесс получения/извлечения информации с веб-ресурсов в интернете называется web-scraping (рус. веб-скрейпинг/веб-скрапинг). Веб-скрапинг может быть проделан вручную пользователем компьютера, однако этот термин обычно связывают с автоматизированными процессами, реализованными с помощью кода.

Какие данные можно извлечь в процессе веб-скрапинга?

- цены на товары конкурентов для оптимизации своей стратегии ценообразования;
- сообщения в социальных медиа, по которым можно отслеживать тренды в той или иной области;
- отзывы о товарах/услугах компании на различных площадках, которые можно впоследствии анализировать;
- контактные данные пользователей соцсетей или форумов для дальнейшего взаимодействия с этими пользователями;
- и т.д.

Для того чтобы веб-скрапинг был возможен, необходимо подключение компьютера, который будет задействован в этом процессе, к сети Интернет.

Интернет — это глобальная информационная сеть, которая позволяет компьютерам по всему миру обмениваться информацией. Один компьютер (называемый клиентом) отправляет запрос в определённом формате другому компьютеру (называемому сервером) и получает ответ (текст, изображение, видео и т. д.).

Клиент и сервер взаимодействуют между собой, обмениваясь одиночными сообщениями (не потоком данных) посредством сетевых протоколов, которые формализуют общение между ними. В настоящее время повсеместно используемый протокол в интернете, позволяющий клиенту получать различные ресурсы (например, HTML-документы), — это протокол HTTP.

Запрос, отправляемый клиентом с использованием протокола HTTP, состоит из нескольких элементов:

- адрес, по которому идёт обращение (например, www.google.com);
- техническая информация, например метод запроса;
- дополнительные данные, например если загружается (передаётся) изображение.
- Адрес — это URL, Uniform Resource Locator (с англ. Унифицированный Указатель Ресурса).

Ответ, в свою очередь, состоит из следующих элементов:

- код статуса ответа: например, 200 («успешно»), 404 («не найден») и т. д. (более полный список кодов статуса ответа можете посмотреть, перейдя по ссылке);
- текст в запрошенном формате (HTML, XML, JSON и т. д.) или мультимедийные файлы;
- прочая техническая информация.

### МЕТОДЫ ЗАПРОСОВ В ПРОТОКОЛЕ HTTP

Для того чтобы указать серверу на то, какое действие мы хотим произвести с ресурсом, в протоколе HTTP используются так называемые методы. В HTTP существует несколько методов, которые описывают действия с ресурсами. Чаще всего используются GET и POST.

#### GET — ПОЛУЧЕНИЕ РЕСУРСА

Метод GET запрашивает информацию из указанного источника и не может влиять на его содержимое. Запрос доступен для кэширования данных (то есть для сохранения, восстановления и дальнейшего использования) и добавления в закладки. Длина запроса ограничена (максимальная длина — 2048 символов).

Пример GET-запроса, отправляемого через адресную строку браузера:

-  http://site.ru/page.php?name=dima&age=27

#### POST — СОЗДАНИЕ РЕСУРСА

Метод POST используется для отправки данных, которые могут оказывать влияние на содержимое ресурса. В отличие от метода GET, запросы POST не могут быть кэшированы, они не остаются в истории браузера и их нельзя добавить в закладки. Длина запроса POST не ограничивается.

Пример POST-запроса, отправляемого через форму запроса:

- POST / HTTP/1.0\r\n
- Host: www.site.ru\r\n
- Referer: http://www.site.ru/index.html\r\n
- Cookie: income=1\r\n
- Content-Type: application/x-www-form-urlencoded\r\n
- Content-Length: 35\r\n
- \r\n
- login=Dima&password=12345

Эти знания пригодятся нам с вами в дальнейшем при подготовке запросов на получение информации от сервера и анализе полученных от него ответов.

# 3. Библиотека requests

✍ В стандартной библиотеке Python для отправки веб-запросов существует функция urllib2, но большинство разработчиков используют стороннюю библиотеку requests (c англ. запросы), потому что её работа более стабильна, а созданный с её помощью код получается проще. Поэтому мы будем работать с библиотекой requests, а urllib2 рассматривать не будем.

Познакомимся с библиотекой requests, решив простую задачу — получить значения курсов валют. Курс валют — полезная и регулярно обновляемая информация, но каждый раз в ручном режиме получать информацию о курсе интересующей валюты трудоёмко.

Разработаем код, так называемый скрипт (англ. script, рус. сценарий), — небольшую программу, которая содержит последовательность действий для автоматического выполнения задачи.

С помощью скрипта мы будем в удобном виде выгружать информацию по курсам валют с заранее выбранного сайта.

- Один из сайтов в интернете, на котором информация о курсах валют дублирует информацию с сайта Центрального Банка России, — ресурс Курсы валют ЦБ РФ в XML и JSON. На данном ресурсе информация о курсах валют представлена в разных форматах, в том числе и в структурированном JSON-формате, методы работы с которым мы изучили в одном из предыдущих модулей.



In [ ]:
# Устанавливаем библиотеку requests
# Напишем в терминале pip install requests 

Как только библиотека установлена, импортируем её и отправим наш первый запрос к ресурсу Курсы валют ЦБ РФ в XML и JSON. Используем метод get() из библиотеки requests, передав ему соответствующий URL —  https://www.cbr-xml-daily.ru/daily_json.js:

In [1]:
import requests # Импортируем библиотеку requests
url = 'https://www.cbr-xml-daily.ru/daily_json.js' # Определяем значение URL страницы для запроса
response = requests.get(url) # Делаем GET-запрос к ресурсу и результат ответа сохраняем в переменной response

In [2]:
# Проверим ответ сервера — содержимое переменной response:
print(response) # Выводим значение response на экран как объект

<Response [200]>


### РАБОТАЕМ С ОТВЕТОМ

Мы сделали запрос и получили корректный ответ (код статуса — 200). Дальнейшую работу производим с результатом запроса к ресурсу Курсы валют ЦБ РФ в XML и JSON.

? - Как получить доступ ко всей информации, которую содержит ответ?

Текст ответа хранится в атрибуте text. Выведем значение атрибута на экран и посмотрим на его содержимое:

In [ ]:
print(response.text) # Выводим содержимое атрибута text переменной response на экран

Как правило, при работе над реальным проектом на этапе получения данных мы уже понимаем, с какими форматами данных нам придётся работать. На предлагаемом для работы ресурсе информация есть как в JSON-формате, так и в XML. По нашему запросу ресурс возвращает информацию в JSON-формате, однако в настоящий момент результат хранится как единая строка. Проверить тип данных полученного ответа можно, воспользовавшись функцией type().

Для того чтобы удобно было работать с полученной информацией, нам необходимо преобразовать строку в словарь. В объект ответа Response  из библиотеки requests уже встроен метод json() .

Импортируем функцию pprint(), применим к полученному ответу метод json() и выведем полученный результат на экран:

In [ ]:
from pprint import pprint # Импортируем функцию pprint()
import json # Импортируем модуль json
currencies = response.json() # Применяем метод json()
pprint(currencies) # Выводим результат на экран

Теперь данные находятся в словаре и можно легко получать необходимые значения.

Например, по ключу Valute мы можем обратиться к вложенному словарю, который содержит информацию о мировых валютах. Выведем на экран, например, информацию о евро (EUR):

In [5]:
pprint(currencies['Valute']['EUR']) # Выводим на экран информацию о валюте евро

{'CharCode': 'EUR',
 'ID': 'R01239',
 'Name': 'Евро',
 'Nominal': 1,
 'NumCode': '978',
 'Previous': 98.3062,
 'Value': 98.9461}


In [6]:
print(currencies['Valute']['CZK']['Name'])

Чешских крон


# 4. Парсинг сайтов
✍ Ресурс, с которым мы работали в предыдущем юните, возвращал ответ о текущем курсе валют в удобном, структурированном формате, из которого было легко извлечь необходимую информацию. Довольно часто для получения информации приходится обращаться напрямую к HTML-страницам.

Для примера рассмотрим страницу, содержащую статью с информацией о присуждении Нобелевской премии по экономике в 2021 году, и попробуем извлечь из неё заголовок статьи, опубликованной на странице, дату публикации, а также текст статьи.

Получить содержимое страницы в большинстве случаев несложно, гораздо труднее извлечь из HTML-кода нужную информацию.

?
Что собой представляет HTML?

### ОСНОВЫ HTML

HTML (англ. HyperText Markup Language, рус. язык гипертекстовой разметки) — стандартизированный язык разметки документов в интернете. Большинство веб-страниц содержат описание разметки на языке HTML. Язык HTML интерпретируется браузерами. Полученный в результате интерпретации текст отображается на экране монитора компьютера или мобильного устройства.

HTML позволяет создавать макет страницы, разбивая её на блоки: мы можем поместить содержимое посередине страницы, сбоку и т. п.

Кроме того, HTML используется для описания форматирования. Например, с его помощью мы можем указать, какая часть текста должна отображаться крупным шрифтом как заголовок, какая — курсивом, а какая — как обычный текст.

HTML является близким родственником уже знакомого вам формата XML. Разметка на языке HTML делается с помощью так называемых тегов, которые помещаются в угловые скобки, и применяется к элементам, заключённым внутри них. Посмотрите на примеры:

In [ ]:
# <h2> Это заголовок второго уровня </h2>
# <div> А это обычный текст </div>

У корректной HTML-страницы есть заголовок и тело страницы. В заголовке (в тегах <head> … </head>)  размещается техническая информация, подключаются скрипты и стили. В теле <body> … </body> находятся текст и данные, которые непосредственно отображаются на странице в браузере.

Разметка небольшой страницы выглядит примерно так:

In [ ]:
#<!DOCTYPE html>
#<html lang="ru">
#    <head>
#        <title>Название страницы</title>
#        <meta charset="UTF-8">
#    </head>
#    <body>
#        <h1> Это заголовок страницы </h1>
#        <p> Какой-то текст </p>
#    </body>
#</html>

Вы можете сохранить этот код в текстовом файле с расширением .html и открыть этот файл в браузере.

Обратите внимание, что теги образуют иерархическую структуру, то есть одни теги расположены внутри других. В примере выше тег <'p> … <'/p> находится внутри тега <'body> … <'/body>.

Кроме того, у тегов могут быть атрибуты, которые пишутся внутри открывающегося тега. Самые популярные атрибуты — это class и id:

In [ ]:
# <h1 id="big-title"> Заголовок страницы </h1>
# <p class="red-back"> Какой-то текст </p>

Изучение языка HTML находится вне рамок этого курса, но для того, чтобы собирать информацию с веб-страниц, нет необходимости хорошо знать HTML. Достаточно понимать, что:

- существуют теги с разными именами;
- у тегов бывают атрибуты, такие как class и id;
- теги образуют иерархическую структуру, то есть одни теги вложены в другие.

### ПОЛУЧАЕМ СОДЕРЖИМОЕ ВЕБ-СТРАНИЦЫ

Получим HTML-код интересующей нас страницы.

Для этого отправим GET-запрос с помощью библиотеки requests и метода get() и посмотрим на текст ответа на наш запрос (как мы помним, он содержится в атрибуте text):

In [ ]:
import requests # Импортируем библиотеку requests
url = 'https://nplus1.ru/news/2021/10/11/econobel2021' # Определяем адрес страницы
response = requests.get(url)  # Выполняем GET-запрос
print(response.text)  # Выводим содержимое атрибута text

Ответ содержит HTML-код страницы, к которой мы обратились.

В отличие от предыдущего примера, где ответ возвращался в JSON-формате, мы не можем так просто преобразовать HTML-код в словарь и извлечь необходимую нам информацию.

Для решения таких задач в Python существует специальная библиотека BeautifulSoup, о работе с которой мы поговорим в следующем юните.

# 5. Библиотека BeautifulSoup

✍ Для поиска необходимых нам данных мы будем использовать библиотеку BeautifulSoup, которая позволяет по названию тегов и их атрибутов получать содержащийся в них текст.

BeautifulSoup не является частью стандартной библиотеки, поэтому для начала её нужно установить. Например, в Jupyter Notebook это делается с помощью такой команды:


In [ ]:
# Устанавливаем библиотеку BeautifulSoup
# pip install beautifulsoup4 

In [8]:
# После установки импортируем библиотеку в наш код:

from bs4 import BeautifulSoup # Импортируем библиотеку BeautifulSoup

Теперь мы можем извлекать данные из любой веб-страницы.

Ранее мы уже получили содержимое страницы с помощью GET-запроса и сохранили информацию в переменной response , теперь создадим объект BeautifulSoup с именем page, указывая в качестве параметра html.parser.

Для примера получим информацию o title (с англ. заголовок) — это строка, которая отображается на вкладке браузера:

In [9]:
import requests # Импортируем библиотеку requests
from bs4 import BeautifulSoup # Импортируем библиотеку BeautifulSoup
url = 'https://nplus1.ru/news/2021/10/11/econobel2021' # Определяем адрес страницы
response = requests.get(url) # Выполняем GET-запрос, содержимое ответа присваивается переменной response
page = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup, указывая html-парсер
print(page.title) # Получаем тег title, отображающийся на вкладке браузера
print(page.title.text) # Выводим текст из полученного тега, который содержится в атрибуте text

<title>Премию Нобеля по экономике присудили за исследования экономики труда и причинно-следственных связей</title>
Премию Нобеля по экономике присудили за исследования экономики труда и причинно-следственных связей


Если при запросе к сайту, а затем при его разборе с помощью BeautifulSoup в тексте страницы не находится нужный тег, попробуйте вывести на печать пару тысяч символов текста страницы. Если там обнаружится нечто похожее на капчу, возможно, сайт посчитал вас роботом и отказывается выдавать содержимое. Чтобы получить его, попробуйте «притвориться» браузером при запросе из скрипта:

In [ ]:
# requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})

### ИЗВЛЕКАЕМ ЗАГОЛОВОК СТАТЬИ

Выполним поставленную ранее задачу: получить информацию о странице и извлечь заголовок статьи, опубликованной на этой странице, дату публикации, а также текст статьи.

Предположим, что мы знаем, что в HTML-коде рассматриваемой нами страницы заголовок статьи заключён в тег <'h1> … <'/h1> (заголовок первого уровня).

Тогда мы можем получить его текст с помощью метода find() (с англ. найти) объекта BeautifulSoup, передав ему название интересующего нас тега:

In [10]:
# Применяем метод find() к объекту и выводим результат на экран
print(page.find('h1').text) 


            Премию Нобеля по экономике присудили за исследования экономики труда и причинно-следственных связей
          


Но как же узнать, в каких именно тегах заключена необходимая информация?

Проще всего это сделать с помощью так называемого инструмента разработчика, который есть во всех современных браузерах. Покажем, как открыть данный инструмент на примере использования браузера Google Chrome.

Устанавливаем курсор на элементе страницы (заголовок статьи), информацию о котором хотим получить, нажимаем на правую клавишу мыши и в выпадающем списке выбираем пункт «Просмотреть код элемента» или «Исследовать» в зависимости от браузера.

В открывшемся окне инструмента разработчика видим, что информация о заголовке статьи заключена в теге <'h1> … <'/h1>.



### Задание 5.4
Напишите функцию wiki_header, которая по адресу страницы возвращает заголовок первого уровня для статей на Wikipedia.

Функция wiki_header принимает один аргумент - url.

In [11]:
import requests
from bs4 import BeautifulSoup

def wiki_header(url):
    page=BeautifulSoup(requests.get(url).text,'html.parser')
    header=page.find('h1').text
    return header

wiki_header('https://en.wikipedia.org/wiki/Operating_system')
'Operating system'



'Operating system'

### НЕУНИКАЛЬНЫЕ ТЕГИ: ИЗВЛЕКАЕМ ТЕКСТ И ДАТУ ПУБЛИКАЦИИ СТАТЬИ

Теперь получим сам текст статьи. Как вы уже знаете, первым делом необходимо определить, в какой тег он заключён. Применим, как и ранее, инструмент разработчика.

Видим, что искомый текст заключён в тег  <'div> … <'/div> . Попробуем извлечь его уже известным нам способом — с помощью метода find() — и выведем его на экран.

In [12]:
print(page.find('div').text) # Выводим содержимое атрибута text тега div

Мы увидели не то, что ожидали — кучу текста, не имеющего отношения к тому, что мы искали...
?
В чём же проблема?

Дело в том, что теги <'div> … <'/div> очень распространённые и на странице их очень много. Метод find() нашёл первый из них, но это не то, что нам надо.

Посмотрим на нашу страницу, используя инструмент разработчика, ещё раз. Можем заметить, что у искомого текста есть свой класс — n1_material text-18 :

Передадим название класса в метод find() с помощью аргумента class_ и получим текст статьи:

In [13]:
print(page.find('div', class_='n1_material text-18').text) # Выводим содержимое атрибута text тега div класса n1_material text-18

Премия Шведского национального банка по экономическим наукам памяти Альфреда Нобеля за 2021 год присуждена Дэвиду Карду (David Card) за его вклад в эмпирические исследования экономики рынка труда, а также Джошуа Энгристу (Joshua Angrist) и Гвидо Имбенсу (Guido Imbens) за их вклад в методологию анализа причинно-следственных связей. Прямая трансляция церемонии объявления лауреатов шла на официальном сайте Нобелевской премии.


В данном случае происходит поиск точного строкового значения class атрибута, т. е. выполнение строк кода:

In [14]:
print(page.find('div', class_='n1_material').text)

Премия Шведского национального банка по экономическим наукам памяти Альфреда Нобеля за 2021 год присуждена Дэвиду Карду (David Card) за его вклад в эмпирические исследования экономики рынка труда, а также Джошуа Энгристу (Joshua Angrist) и Гвидо Имбенсу (Guido Imbens) за их вклад в методологию анализа причинно-следственных связей. Прямая трансляция церемонии объявления лауреатов шла на официальном сайте Нобелевской премии.


In [15]:
print(page.find('div', class_='n1_material text-18').text)
# даст одинаковый результат.

Премия Шведского национального банка по экономическим наукам памяти Альфреда Нобеля за 2021 год присуждена Дэвиду Карду (David Card) за его вклад в эмпирические исследования экономики рынка труда, а также Джошуа Энгристу (Joshua Angrist) и Гвидо Имбенсу (Guido Imbens) за их вклад в методологию анализа причинно-следственных связей. Прямая трансляция церемонии объявления лауреатов шла на официальном сайте Нобелевской премии.


In [ ]:
#При выполнении строки кода

print(page.find('div', class_='text-18 n1_material').text)
#мы получим ошибку, так как такого строкового значения в области поиска нет.

Аналогично получим информации о теге, который содержит дату написания статьи, отображаемую в левом верхнем углу страницы.

Итак, нам нужен тег <'a> … <'/a> с классом "relative before:block before:w-px before:bg-current before:h-4 before:absolute before:left-0 group pl-2 flex inline-flex items-center". Для поиска достаточно указать в качестве класса "relative", отбросив дополнительные настройки.

Теперь получим данные из него с помощью уже известного метода find(), передав название нужного тега:

In [17]:
# Выводим на экран содержимое атрибута text тега a с классом "relative"
print(page.find('a', class_= "relative").text)


11.10.21



О поиске по классу можно узнать подробнее в Beautiful Soup Documentation.

Задача решена — мы извлекли из контента страницы заголовок статьи, опубликованной на странице, дату публикации, а также текст статьи.

### СБОР НЕСКОЛЬКИХ ЭЛЕМЕНТОВ: СОБИРАЕМ ВСЕ ССЫЛКИ НА СТРАНИЦЕ

Рассмотрим ещё один сценарий: вы хотите собрать сразу несколько элементов со страницы. Например, представьте, что вы хотите получить названия всех языков программирования, упомянутых на странице в Wikipedia в статье про языки программирования.

Можно заметить, что все названия языков программирования на этой странице связаны ссылками c соответствующими статьями о них. Таким образом, нам необходимо собрать все ссылки на странице. Для ссылок в HTML предусмотрен тег <'a> … <'/a>. Попробуем использовать find():

In [18]:
url = 'https://en.wikipedia.org/wiki/List_of_programming_languages' # Задаём адрес ресурса
response = requests.get(url) # Делаем GET-запрос к ресурсу
page = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup
print(page.find('a')) # Ищем ссылку по тегу <a> и выводим её на экран

<a class="mw-jump-link" href="#bodyContent">Jump to content</a>


Мы получили только одну ссылку, хотя на странице их явно больше.

Это происходит, потому что метод find() возвращает только первый подходящий элемент. Если требуется получить больше элементов, необходимо воспользоваться методом find_all() (с англ. найти все):

In [19]:
links = page.find_all('a') # Ищем все ссылки на странице и сохраняем в переменной links в виде списка
print(len(links)) # Выводим количество найденных ссылок
# Итак, на момент создания этих учебных материалов на странице содержалось 928 ссылок. Посмотрим на некоторые из них:

959


In [20]:
print([link.text for link in links[500:510]]) # Выводим ссылки с 500 по 509 включительно

['M#', 'Machine code', 'MAD', 'MAD/I', 'Magik', 'Magma', 'Maple', 'MAPPER', 'MARK-IV', 'Mary']


Не все ссылки соответствуют названиям языков программирования — страница содержит также «служебные» ссылки, такие, например, как Jump to navigation (с англ. Перейти к навигации) или Alphabetical (с англ. По алфавиту):

In [21]:
print([link.text for link in links[0:10]]) # Выводим ссылки с 1 по 9 включительно

['Jump to content', 'Main page', 'Contents', 'Current events', 'Random article', 'About Wikipedia', 'Contact us', 'Donate', 'Help', 'Learn to edit']


Для обработки полученных данных и исключения «лишней» информации можно, например, использовать подходы, которые вы изучили в модуле PYTHON-14.

✍ В заключение заметим, что BeautifulSoup — достаточно мощная библиотека. Мы рассмотрели её базовые возможности, но их полный список гораздо шире. С ним можно ознакомиться в официальной документации.

# 6. Работа с API
✍ В предыдущих разделах мы собирали полезную информацию с различных сайтов.

Вы могли убедиться, что поиск необходимой информации с выделением правильных тегов — довольно трудоёмкая задача. Кроме того, подобные программы могут ломаться в случаях, когда меняется дизайн сайта, его разметка или владельцы сайтов защищаются от ботов капчей.

К счастью, многие крупные сайты предоставляют доступ к так называемым API (англ. Application Programming Interface, рус. Интерфейс Прикладного Программирования).

API — это специальные разделы сайта, где информацию можно получать без разметки, а формат запросов и ответов зафиксирован. API созданы для того, чтобы облегчить взаимодействие с сайтом для сторонних разработчиков.

К примеру, мы уже видели, как ресурс Курсы валют ЦБ РФ в XML и JSON возвращает данные о валютах в JSON-формате. Это пример API.

Рассмотрим на примере социальной сети ВКонтакте особенности API, характерные для более крупных сайтов.

### КЛЮЧ АВТОРИЗАЦИИ

Для того чтобы начать работать с API, обычно необходимо получить сервисный ключ авторизации — токен.

Токен — это средство идентификации пользователя или отдельного сеанса работы в компьютерных сетях и приложениях. Различают программные и аппаратные токены.
Мы будем использовать программный токен, который обычно представляет собой зашифрованную последовательность символов, позволяющую точно идентифицировать объект и определить уровень его привилегий. Он генерируется системой авторизации и привязывается к конкретному сеансу работы, клиенту сети или пакету данных.

Авторизация применяется практически во всех API, чтобы отдавать данные только их владельцу или контролировать количество запросов в единицу времени.

Сервисный токен для API ВКонтакте для нашей задачи создаётся вместе с новым приложением. Приложение мы делать, конечно, не будем. Оно нужно только для получения токена, чтобы сделать необходимые выгрузки.

Зайдите на страницу, чтобы создать приложение (вы должны быть авторизованы ВКонтакте). Дайте приложению любое название и в разделе Платформа поставьте отметку выбора напротив значения "встраиваемое приложение":

### ПЕРВЫЕ ЗАПРОСЫ К API

Чтобы познакомиться с работой API, мы будем получать данные для статистических отчётов произвольной группы, например данные о соотношении мужчин и женщин, статистику географии пользователей и т. п.

Сначала рассмотрим работу API на простом примере, на основе которого работают многие системы.

Сделаем наш первый запрос из браузера.

Перейдите по следующей ниже ссылке в браузере, подставив вместо слова TOKEN ваш персональный сервисный ключ доступа (токен), полученный на предыдущем шаге:

In [ ]:
# https://api.vk.com/method/users.get?user_id=1&v=5.95&access_token=TOKEN
# token = 7dd631217dd631217dd63121de7ece7d7377dd67dd631211b9258489af4446ebd362c82
# {"response":[{"id":1,"first_name":"Павел","last_name":"Дуров","can_access_closed":true,"is_closed":false}]}

Результат:

Итак, мы сделали GET-запрос к API ВКонтакте, который состоит из следующих элементов:

- https://api.vk.com/method — домен и URL запроса API; обычно не меняется;
- users.get — название метода, который отдаёт определённый отчёт, в нашем случае это метод для получения информации о пользователе;
- user_id и v — параметры запроса: идентификатор пользователя, о котором хотим получить информацию (в нашем примере мы запрашиваем информацию о первом пользователе), и номер версии API;
- token — токен, который выдаётся только пользователям, имеющим право просматривать определённые данные, например показания счётчиков Яндекс.Метрики вашего проекта; на все остальные запросы без корректного токена система отвечает отказом.

Если мы обратимся к документации метода users.get, то увидим, что в ней описано множество других параметров, которые можно получить о пользователе (дата рождения, пол, родной город и другие) — словом, всё то, что мы видим на странице пользователя в интерфейсе или приложении ВКонтакте (конечно, если пользователь их указал).

Добавим к запросу дату рождения и пол (согласно документации, эти параметры надо перечислять в поле fields):

In [ ]:
# https://api.vk.com/method/users.get?user_id=1&v=5.95&fields=sex,bdate&access_token=TOKEN
# {"response":[{"id":1,"bdate":"10.10.1984","sex":2,"first_name":"Павел",
# "last_name":"Дуров","can_access_closed":true,"is_closed":false}]}

Примечание: значение 2 у параметра sex означает мужской пол.

До этого момента мы делали запросы в браузере — теперь давайте выполним запросы из кода.

### ЗАПРОС К API ИЗ КОДА

Продолжаем пользоваться всё той же библиотекой requests.

In [23]:
import requests # Импортируем модуль requests
token = '7dd631217dd631217dd63121de7ece7d7377dd67dd631211b9258489af4446ebd362c82' # Указываем свой сервисный токен
url = 'https://api.vk.com/method/users.get' # Указываем адрес страницы к которой делаем запрос
params = {'user_id': 1, 'v': 5.95, 'fields': 'sex,bdate', 'access_token': token, 'lang': 'ru'} # Перечисляем параметры нашего запроса в словаре params
response = requests.get(url, params=params) # Отправляем запрос
print(response.text) # Выводим текст ответа на экран

{"response":[{"id":1,"bdate":"10.10.1984","sex":2,"first_name":"Павел","last_name":"Дуров","can_access_closed":true,"is_closed":false}]}


Мы получили строку в JSON-формате, которую можно преобразовать в словарь с помощью метода json(), после чего можно с лёгкостью обращаться к различным полям.

Словари нагляднее выводить с помощью функции pprint(), которую мы уже использовали ранее:

In [24]:
from pprint import pprint # Импортируем функцию pprint()
pprint(response.json()) # Выводим содержимое словаря, содержащего ответ, на экран

{'response': [{'bdate': '10.10.1984',
               'can_access_closed': True,
               'first_name': 'Павел',
               'id': 1,
               'is_closed': False,
               'last_name': 'Дуров',
               'sex': 2}]}


Как вы видите, по ключу response мы можем получить список, в котором хранятся словари, содержащие информацию о запрошенных нами пользователях. Мы запросили информацию лишь об одном из них, поэтому список содержит только один элемент. Извлечём его:

In [25]:
user = response.json()['response'][0] # Извлекаем из словаря по ключу response информацию о первом пользователе
print(user['bdate']) # Выводим дату рождения первого пользователя на экран

10.10.1984


Метод users.get() позволяет запрашивать информацию о множестве (до 1 000) пользователей одновременно. Для этого нужно использовать параметр user_ids и передавать id через запятую в строковом формате. Например, чтобы получить информацию о пользователях с id=1, id=2, id=3, необходимо передать значение параметра user_ids='1,2,3'.

Попробуем это сделать:

In [26]:
ids = ",".join(map(str, range(1, 4))) # Формируем строку, содержащую информацию о поле id первых трёх пользователей
params = {'user_ids': ids, 'v': 5.95, 'fields': 'bdate', 'access_token': token, 'lang': 'ru'} # Формируем строку параметров
pprint(requests.get(url, params=params).json()) # Посылаем запрос, полученный ответ в формате JSON-строки преобразуем в словарь и выводим на экран его содержимое, используя функцию pprint()


{'response': [{'bdate': '10.10.1984',
               'can_access_closed': True,
               'first_name': 'Павел',
               'id': 1,
               'is_closed': False,
               'last_name': 'Дуров'},
              {'bdate': '14.2',
               'can_access_closed': False,
               'first_name': 'Александра',
               'id': 2,
               'is_closed': True,
               'last_name': 'Владимирова'},
              {'can_access_closed': True,
               'deactivated': 'deleted',
               'first_name': 'DELETED',
               'id': 3,
               'is_closed': False,
               'last_name': ''}]}


### Задание 6.2

Используя API, определите долю женщин (sex=1) среди пользователей с id от 1 до 500. Иногда будут попадаться пользователи, у которых пол не указан (sex=0), — таких пользователей не нужно учитывать в общем числе.

В ответе укажите число, округлив до двух знаков после точки-разделителя, например, 0.55.

Пример: если у нас будет 300 пользователей с sex=1, 100 пользователей с sex=2 и 100 пользователей с sex=0, то в ответе должно быть 0.75.

In [33]:
ids = ",".join(map(str, range(1, 501)))  # Формируем строку, содержащую информацию о поле id первых 500 пользователей
params = {'user_ids': ids, 'v': 5.95, 'fields': 'sex', 'access_token': token, 'lang': 'ru'}

# Отправляем запрос
response = requests.get(url, params=params)
data = response.json()

# Обработка ответа
if 'response' in data:
    users = data['response']
    total_users = len(users)
    female_count = sum(1 for user in users if 'sex' in user and user['sex'] == 1)
    male_count = sum(1 for user in users if 'sex' in user and user['sex'] == 2)
    female_ratio = female_count / (male_count+female_count)
    print(round(female_ratio, 2))
    

0.53


### СБОР ИНФОРМАЦИИ ИЗ ГРУПП

В одном из предыдущих юнитов в качестве примера мы собрали информацию о небольшом количестве пользователей. Теперь перейдём к более реальной задаче — сбору данных о пользователях группы ВКонтакте.

Стоит отметить, что есть много сервисов, которые выгружают похожую статистику из соцсетей. Однако им свойственны недостатки универсальных решений:

- не учитываются все особенности вашего проекта;
- используется фиксированный набор метрик, дополнительную обработку данных приходится делать вам;
- не всегда бесплатны и вряд ли позволят работать с большими объёмами данных.

Теперь мы научимся считать произвольные метрики групп, собирая данные из API и работая с двумя ограничениями, которые свойственны практически всем системам:

- ограничение на количество вызовов в единицу времени;
- ограничение на количество выгружаемых строк за один запрос.

Ограничение на количество запросов в единицу времени сделано для того, чтобы избежать чрезмерной нагрузки на серверы системы. В ряде случаев небольшое количество отчётов можно выгрузить и уложиться в действующий лимит.

Однако второе ограничение не удастся обойти в случае выгрузки больших отчётов. Например, чтобы получить список всех пользователей популярной группы, серверу, возможно, придётся отправить ответ, содержащий миллионы записей.



Давайте рассмотрим, как работать с этими ограничениями на примере выгрузки списка пользователей группы https://vk.com/vk социальной сети ВКонтакте.

Обратимся к документации, чтобы узнать, какие методы нам доступны для групп, — для получения списка пользователей группы доступен метод groups.getMembers.

Согласно документации, обязательным параметром данного метода является group_id — идентификатор, или короткое имя, группы. В нашем случае это vk: https://vk.com/vk. Протестируем, как работает метод в самом простом случае, — получим id участников группы:

In [38]:
import requests # Импортируем модуль requests
token = '7dd631217dd631217dd63121de7ece7d7377dd67dd631211b9258489af4446ebd362c82' # Указываем свой сервисный токен
url = 'https://api.vk.com/method/groups.getMembers' # Указываем адрес обращения
params = {'group_id': 'vk', 'v': 5.95, 'access_token': token} # Формируем строку параметров
response = requests.get(url, params = params) # Посылаем запрос
data = response.json() # Ответ сохраняем в переменной data в формате словаря
print(data) # Выводим содержимое переменной data на экран (отображён фрагмент)

{'response': {'count': 13529565, 'items': [6, 19, 47, 54, 79, 177, 198, 212, 219, 239, 243, 296, 345, 421, 431, 450, 467, 485, 510, 550, 619, 640, 690, 702, 721, 804, 809, 831, 832, 834, 847, 900, 905, 907, 914, 930, 943, 952, 958, 966, 976, 997, 1000, 1018, 1023, 1032, 1033, 1038, 1039, 1059, 1097, 1131, 1139, 1140, 1159, 1174, 1185, 1188, 1301, 1333, 1334, 1351, 1381, 1386, 1388, 1406, 1411, 1418, 1432, 1494, 1503, 1531, 1550, 1568, 1586, 1590, 1593, 1598, 1610, 1615, 1632, 1634, 1650, 1679, 1690, 1697, 1698, 1699, 1700, 1721, 1740, 1754, 1796, 1814, 1820, 1829, 1834, 1839, 1840, 1843, 1858, 1863, 1869, 1887, 1889, 1917, 1943, 1947, 1955, 1969, 2019, 2028, 2050, 2051, 2052, 2059, 2077, 2103, 2136, 2150, 2195, 2201, 2230, 2236, 2273, 2281, 2296, 2298, 2376, 2389, 2395, 2403, 2412, 2436, 2456, 2466, 2470, 2484, 2513, 2515, 2539, 2571, 2576, 2592, 2601, 2622, 2644, 2654, 2692, 2706, 2745, 2755, 2767, 2787, 2797, 2827, 2858, 2909, 2919, 2922, 2932, 2953, 2962, 2982, 3001, 3006, 3041, 313

По ключу count мы можем получить общее число участников группы, а список по ключу items хранит их id. Посмотрим на него поближе:

In [39]:
print(len(data['response']['items'])) # Выводим на экран количество элементов словаря

1000


Мы видим, что всего пользователей в группе больше 11 миллионов, а получили мы только первую тысячу пользователей группы. По информации, указанной в документации о параметре count, это максимум, который может отдать API за один раз.

Для получения следующей тысячи пользователей можно воспользоваться параметром offset (с англ. смещение), который передвинет начало отсчёта. Для выгрузки всех пользователей группы будем в цикле выгружать по 1000 пользователей (count будет всегда равен 1000), увеличивая смещение offset на величину count.

Для тренировки напишем цикл выгрузки первых 20 пользователей со значением count=5. Иными словами, мы будем выгружать по пять пользователей за запрос до тех пор, пока не получим информацию о 20 пользователях.

Давайте выведем на экран первые 20 пользователей из нашей первой попытки получить информацию о 1000 пользователей, чтобы мы могли сверить результат выгрузки из 20 пользователей:

In [40]:
users_for_checking = data['response']['items'][:20] # Загружаем в переменную информацию об id первых 20 пользователей в виде списка
print(users_for_checking) # Выводим перечень id первых 20 пользователей

[6, 19, 47, 54, 79, 177, 198, 212, 219, 239, 243, 296, 345, 421, 431, 450, 467, 485, 510, 550]


In [42]:
# Теперь используем count и offset, чтобы получить те же id по пять за раз:

import requests # Импортируем модуль requests
token = '7dd631217dd631217dd63121de7ece7d7377dd67dd631211b9258489af4446ebd362c82' # Указываем свой сервисный токен
url = 'https://api.vk.com/method/groups.getMembers' # Указываем адрес обращения
count = 5 
offset = 0 
user_ids = [] 
max_count = 20 
while offset < max_count: 
    # Будем выгружать по count=5 пользователей, 
    # начиная с того места, где закончили на предыдущей итерации (offset) 
    print('Выгружаю {} пользователей с offset = {}'.format(count, offset))   
    params = {'group_id': 'vk', 'v': 5.95, 'count': count, 'offset': offset, 'access_token': token} 
    response = requests.get(url, params = params) 
    data = response.json() 
    user_ids += data['response']['items'] 
    # Увеличиваем смещение на количество строк, которое мы уже выгрузили 
    offset += count 
print(user_ids) 

Выгружаю 5 пользователей с offset = 0
Выгружаю 5 пользователей с offset = 5
Выгружаю 5 пользователей с offset = 10
Выгружаю 5 пользователей с offset = 15
[6, 19, 47, 54, 79, 177, 198, 212, 219, 239, 243, 296, 345, 421, 431, 450, 467, 485, 510, 550]


In [43]:
# Сравним списки, полученные двумя способами:

print(user_ids == users_for_checking) 

True


Так как результат сравнения — True, списки идентичны. Значит, второй способ работает корректно. Теперь мы можем получить данные обо всех пользователях, выставив count = 1000 и max_count = data['response']['count'].

### ОГРАНИЧЕНИЕ ПО ЧАСТОТЕ ЗАПРОСОВ

→ В API часто добавляют ограничение по частоте запросов, чтобы отдельно взятые пользователи слишком сильно не перегружали сервер. Подобное ограничение есть и у ВКонтакте — в документации указано, что можно делать не более трёх запросов в секунду.

Чтобы не следить за частотой отправки запросов с секундомером в руках, мы можем после каждого запроса делать паузу. В этом случае, даже если код будет выполняться на самом быстром компьютере, мы не нарушим установленное ограничение, так как периодичность отправки запросов будет искусственно замедлена.

Воспользуемся библиотекой time и методом sleep, с помощью которого мы можем добавить паузу, например в 0.5 секунд, после каждого запроса:

In [44]:
import requests # Импортируем модуль requests
import time # Импортируем модуль time
token = '7dd631217dd631217dd63121de7ece7d7377dd67dd631211b9258489af4446ebd362c82' # Указываем свой сервисный токен
url = 'https://api.vk.com/method/groups.getMembers' # Указываем адрес страницы, к которой делаем запрос
count = 1000 
offset = 0  
user_ids = []  
while offset < 5000: 
    params = {'group_id': 'vk', 'v': 5.95, 'count': count, 'offset': offset, 'access_token': token} 
    response = requests.get(url, params = params) 
    data = response.json() 
    user_ids += data['response']['items'] 
    offset += count 
    print('Ожидаю 0.5 секунды...') 
    time.sleep(0.5) 
print('Цикл завершен, offset =',offset) 

Ожидаю 0.5 секунды...
Ожидаю 0.5 секунды...
Ожидаю 0.5 секунды...
Ожидаю 0.5 секунды...
Ожидаю 0.5 секунды...
Цикл завершен, offset = 5000


### ЛАЙКИ, РЕПОСТЫ И КОММЕНТАРИИ

Через API новостной ленты ВКонтакте мы можем получить информацию о взаимодействии с сообщениями в ленте.

Для примера продолжим работать с группой https://vk.com/vk и рассмотрим последние 100 сообщений в новостной ленте.

Примечание: обратите внимание, что т.к. сообщения в новостной ленте непрерывно обновляются, то ваш результат выполнения кода ниже будет отличаться от нашего варианта.

Для получения информации о сообщениях на стене в API ВКонтакте предусмотрен метод wall.get. Применим его:

In [ ]:
import requests # Импортируем модуль requests
from pprint import pprint # Импортируем функцию pprint()
token = '7dd631217dd631217dd63121de7ece7d7377dd67dd631211b9258489af4446ebd362c82' # Указываем свой сервисный токен
url = 'https://api.vk.com/method/wall.get' # Указываем адрес страницы, к которой делаем запрос
params = {'domain': 'vk', 'filter': 'owner', 'count': 1000, 'offset': 0, 'access_token': token, 'v': 5.95} 
response = requests.get(url, params = params) 
pprint(response.json())

In [46]:
# Посмотрим на количество результатов:

len(response.json()['response']['items'])
## 100

100

In [47]:
# Посмотрим на информацию об отдельном сообщении:

response.json()['response']['items'][0] 

{'inner_type': 'wall_wallpost',
 'comments': {'count': 123},
 'marked_as_ads': 0,
 'hash': '57zn3vq-dStJRPwCFA',
 'type': 'post',
 'attachments': [{'type': 'photo',
   'photo': {'album_id': -7,
    'date': 1715787814,
    'id': 457350275,
    'owner_id': -22822305,
    'access_key': 'e86a7af70d7be57cd1',
    'post_id': 1486494,
    'sizes': [{'height': 73,
      'type': 'm',
      'width': 130,
      'url': 'https://sun6-22.userapi.com/impg/xxgBWuRPF5_kHhJdim-Sm4Jc7KJE_lwGJLHR6w/9gGnSBOrsqM.jpg?size=130x73&quality=96&sign=6d9ab31ce0c707e4c457a3091bbb2a9c&c_uniq_tag=PF2H3EWquagivf7AIGEnJEcIKn9DqEIVf0X9COtD2gE&type=album'},
     {'height': 87,
      'type': 'o',
      'width': 130,
      'url': 'https://sun6-22.userapi.com/impg/xxgBWuRPF5_kHhJdim-Sm4Jc7KJE_lwGJLHR6w/9gGnSBOrsqM.jpg?size=130x87&quality=96&crop=204,0,2152,1440&sign=8a10aa93d8dde30cc832c67a67a69063&c_uniq_tag=Epfi1lyl17h8EM8HH2CMb_1mFUNdoyyaXm85kaKPsbY&type=album'},
     {'height': 133,
      'type': 'p',
      'width': 200

В полях comments, likes и reposts содержится статистика по взаимодействию с сообщением пользователей (на момент получения информации) — число комментариев, лайков и репостов.

### итоговая статистика
Давайте соберём итоговую статистику для последних десяти непустых сообщений в словарь stats. В качестве ключа будем использовать начало сообщения (если начало сообщения пустое, то информацию о таком сообщении проигнорируем), в качестве значения — список с тремя интересующими нас метриками и временем публикации (комментарии, лайки, репосты, дата публикации):

In [48]:
stats = {} 
count_post = 0 # Счётчик «непустых» сообщений
for record in response.json()['response']['items'][:]:
    title = record['text'][:30] 
    if title: 
        stats[title] = [record['comments']['count'], record['likes']['count'], record['reposts']['count'], record['date']] 
        count_post += 1 
    if count_post < 10: 
        continue 
    else: 
        break 
pprint(stats)

{'Вы наверняка знаете эти треки,': [159, 776, 306, 1714208404],
 'Желаем всем оказаться в такой ': [131, 1273, 67, 1715700893],
 'Мы с обнадёживающим уведомлени': [350, 1100, 203, 1713855660],
 'На дворе длинные выходные, поэ': [415, 718, 113, 1714291201],
 'Негласное правило ВКонтакте № ': [339, 1618, 311, 1715004002],
 'Сервис ВКонтакте, который може': [330, 391, 114, 1715183400],
 'Собираем общий плейлист в комм': [3531, 1426, 295, 1714114801],
 'Теперь ВКонтакте можно записат': [123, 229, 58, 1715787814],
 'Узнали, согласны?': [380, 1430, 155, 1714723201],
 'Чтобы отправиться в прошлое, н': [260, 845, 302, 1713593040]}


Мы рассмотрели базовое взаимодействие с пользователями и группами. ВКонтакте предоставляет достаточно широкие возможности в своём API: всё, что можно делать вручную через браузер, доступно и в API.

### ДРУГИЕ API

Вы познакомились с интерфейсами прикладного программирования — API (на примере API социальной сети ВКонтакте).

API для разработчиков предоставляют и многие другие платформы. Вот список, пожалуй, самых популярных из них:

- Google Maps API
- YouTube API
- Twitter API
- Facebook API

Вы также можете воспользоваться интернет-поиском, указав в поисковой строке, например, «курсы валют API» или «прогноз погоды api», — среди первых результатов выдачи чаще всего с лёгкостью можно найти ссылки на необходимый функционал.



# 7. Как настроить регулярную выгрузку данных
✍ Как уже говорилось, скриптом принято называть небольшую компьютерную программу, которая автоматизирует выполнение некоторой задачи. Программы, которые мы создаём на языке Python, также являются скриптами.

Скрипт можно использовать разными способами:

- Если вам нужно выполнить код один раз или выполнять его очень редко (от случая к случаю), вы можете запустить скрипт вручную из командной строки.
- Если вы, например, написали на Python серверную часть веб-сайта и хотите, чтобы ваш код мог в любое время обрабатывать запросы, вы захотите запустить его как фоновый процесс.

?
А что делать, если вам нужно, чтобы ваш скрипт запускался иногда? Каждую пятницу, 13-го? В день рождения супруга (или супруги)? Или просто каждый час?

В этом случае вам нужен автоматический запуск скриптов, или, как часто его называют программисты, запуск по крону — от английского акронима Cron (англ. Command Run ON) — названия системы для автоматического запуска программ и скриптов на сервере в определённое время.

Автоматический запуск может понадобиться, например:

- если вы хотите с определённой периодичностью скачивать новую информацию с сайтов, например выполнять парсинг новостей для последующего анализа (как мы уже знаем, этот процесс называется web-scraping);
- если ваш скрипт должен следить за курсом акций и каждую минуту делать запрос по API, чтобы получить новые котировки;
- если вы написали обучающую платформу и вам нужно каждый час проверять, кто из студентов приступил к занятиям и насколько успешно продвигается их обучение;
- если у вас есть 500 вендинговых автоматов по продаже солнечных очков и каждые пять минут вы должны опрашивать все автоматы, чтобы узнать, не закончились ли очки.

Примеров может быть очень много. В каждом, даже небольшом, проекте обычно есть 5-10 скриптов, которые с определённой периодичностью обновляют сертификаты доменов, очищают место на диске, считают статистику, генерируют отчёты и так далее. Давайте научимся настраивать запуск этих скриптов в соответствии с задачами, которые стоят перед вами.

### КАК НАСТРОИТЬ АВТОМАТИЧЕСКИЙ ЗАПУСК

Во всех операционных системах есть встроенные средства запуска программ по расписанию. Однако можно задать расписание запуска необходимых вам функций на языке Python внутри скрипта.

Исполняемый по расписанию код часто называют задачей (англ. task). Для планирования задач в Python есть несколько библиотек, среди которых — популярный и простой в использовании модуль schedule (c англ. расписание). Он позволяет запускать код как с определённым интервалом, так и в заданное время.

Модуль schedule не входит в стандартную библиотеку Python, поэтому его необходимо установить:

In [ ]:
# # Устанавливаем библиотеку schedule
# !pip install schedule 

In [49]:
# Для того чтобы у нас появилась возможность использовать модуль в коде, импортируем его:

import schedule # Импортируем модуль schedule

### ПОСТАНОВКА ЗАДАЧИ

Рассмотрим вариант автоматического запуска простой функции, которая выводит на экран короткое сообщение:

In [50]:
def task(): 
    print('Hello! I am a task!') 
    return 

Предположим, что мы хотим, чтобы функция запускалась через определённые интервалы времени, а именно каждые 15 минут.

Для запуска задачи через определённые интервалы времени в модуле schedule используется метод every(), который получает в качестве единственного аргумента число, указывающее, как часто следует запускать код.

Далее вызывается метод, определяющий единицы измерения промежутков времени, через которые будет выполняться функция. В нашем примере это минуты. Вот как будет выглядеть итоговый код:

In [ ]:
schedule.every(15).minutes.do(task)

#Если бы мы хотели запускать задачу, например, каждый час, то могли бы написать:
schedule.every(1).hour.do(task) 

### ВЫПОЛНЕНИЕ ФУНКЦИИ

После того как мы создали нашу функцию и определились со временем её запуска, мы можем запустить наш менеджер расписания (schedule). Для этого надо создать бесконечный цикл. Да-да, именно так, ведь скрипт должен выполняться постоянно, чтобы постоянно проверять, не пришло ли время снова выполнить задачу.

Внутри цикла мы будем вызывать особый метод run_pending() для объекта schedule, который будет проверять, нет ли задачи, которую пора выполнить.

После вызова метода run_pending() нужно будет сделать небольшую паузу, после которой можно будет снова проверять, не пришло ли время для выполнения какой-либо функции.

Давайте напишем этот код.

Для создания паузы мы будем использовать метод sleep из модуля time, поэтому наш код начнётся с импорта данного модуля:

In [ ]:
import time 
while True: 
    schedule.run_pending() 
    time.sleep(1)

Всё готово — теперь можно сохранить файл и выполнить код.

Вот какой код в итоге получился:

In [ ]:
import schedule

def task(): 
    print('Hello! I am a task!') 
    return

schedule.every(15).seconds.do(task)

import time 
while True: 
    schedule.run_pending() 
    time.sleep(1)

Этот код будет каждую секунду проверять, не надо ли выполнить какую-то задачу, и раз в 15 минут будет выводить на экран фразу: "Hello! I am a task!" Вывод сообщения будет повторяться до тех пор, пока вы не остановите выполнение скрипта.